In [1]:
import os
import sys
sys.path.append("/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD") # access submodules
sys.path = [p for p in sys.path if not p.startswith('/local/scratch') or p.startswith('/local/scratch/js2173')]

In [2]:
from data.data_loader import CreateDataLoader
from models.models import create_model

from options.train_options import TrainOptions
from options.test_options import TestOptions
import util.util as util

In [3]:
import torch
from torch.autograd import Variable

In [4]:
import numpy as np
# for testing images in here
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [5]:
# this can only be run once since we're in ipython rather than from an actual command line to parse args from
opt = TestOptions()
print(sys.argv)
tmp = sys.argv
sys.argv = [tmp[0]]
opt = opt.parse(save=False)
sys.argv = tmp

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', '-f', '/run/user/3879/jupyter/kernel-92340d76-1ca7-40a9-9e16-072abe703f3a.json']
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
dataroot: ./datasets/cityscapes/
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 50
instance_feat: False
isTrain: False
label_feat: False
label_nc: 35
loadSize: 1024
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: label2city
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: latest
-------------- End ----------------


In [11]:
train_opt = TrainOptions()
print(sys.argv)
tmp = sys.argv
sys.argv = [tmp[0]]
train_opt = train_opt.parse()
sys.argv = tmp

train_opt.instance_feat = True # put in features to steer generation
train_opt.name = 'label2city_512p_feat'
train_opt.dataroot = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/datasets/cityscapes/'
train_opt.checkpoints_dir = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/checkpoints/'
train_opt.results = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/results/'
#train_opt.no_flip = True



# get test images
train_data_loader = CreateDataLoader(train_opt)
train_dataset = train_data_loader.load_data()



['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', '-f', '/run/user/3879/jupyter/kernel-92340d76-1ca7-40a9-9e16-072abe703f3a.json']
------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
cos_decay: False
cos_decay_update_iters: 100
dataroot: ./datasets/cityscapes/
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
instance_feat: False
isTrain: True
label_feat: False
label_nc: 35
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: label2city
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
p

In [ ]:

# # filename we're looking for
# cobblestone_road_image = "jena_000034_000019"
# cobblestone_data = None
# for (i, data) in enumerate(train_dataset):
#     if i % 100 == 0:
#         print("Scanning file number: ", i)
#     if cobblestone_road_image in data['path'][-1]:
#         print("Found cobblestone image!")
#         cobblestone_data = data
#         break

In [ ]:
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.instance_feat = True # put in features to steer generation
opt.name = 'label2city_512p_feat'
opt.dataroot = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/datasets/cityscapes/'
opt.checkpoints_dir = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/checkpoints/'
opt.results = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/results/'


# get test images
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
# create model
model = create_model(opt)

In [ ]:
datas = enumerate(dataset)
print(dataset.dataset.dataset_size)
n, im0 = next(datas)
label_map = Variable(im0['label'])
size = label_map.size()
oneHot_size = (size[0], opt.label_nc, size[2], size[3])
input_label = torch.cuda.FloatTensor(torch.Size(oneHot_size)).zero_()
input_label_2 = input_label.scatter_(1, label_map.data.long().cuda(), 1.0)


In [ ]:
labels = {
0: 'unlabeled',
1: 'ego vehicle',
2: 'rectification border',
3: 'out of roi',
4:'static',
5:'dynamic' ,
6:'ground',
7:'road',
8:'sidewalk',
9:'parking',
10:'rail track',
11: 'building',
12:'wall',
13:'fence',
14:'guard rail',
15:'bridge',
16:'tunnel',
17:'pole',
18:'polegroup',
19:'traffic light',
20:'traffic sign',
21:'vegetation',
22:'terrain',
23:'sky',
24:'person',
25:'rider',
26:'car',
27:'truck',
28:'bus',
29:'caravan',
30:'trailer',
31:'train',
32:'motorcycle',
33:'bicycle',
-1: 'license plate'
}

lab_to_id = { 
'unlabeled' : 0 ,
 'ego vehicle' : 1 ,
 'rectification border' : 2 ,
 'out of roi' : 3 ,
'static' : 4 ,
'dynamic'  : 5 ,
'ground' : 6 ,
'road' : 7 ,
'sidewalk' : 8 ,
'parking' : 9 ,
'rail track' : 10 ,
 'building' : 11 ,
'wall' : 12 ,
'fence' : 13 ,
'guard rail' : 14 ,
'bridge' : 15 ,
'tunnel' : 16 ,
'pole' : 17 ,
'polegroup' : 18 ,
'traffic light' : 19 ,
'traffic sign' : 20 ,
'vegetation' : 21 ,
'terrain' : 22 ,
'sky' : 23 ,
'person' : 24 ,
'rider' : 25 ,
'car' : 26 ,
'truck' : 27 ,
'bus' : 28 ,
'caravan' : 29 ,
'trailer' : 30 ,
'train' : 31 ,
'motorcycle' : 32 ,
'bicycle' : 33 ,
 'license plate' : -1
}

def custom_inference(pix2pixHD, label, inst, feat_map):
    # Encode Inputs        
    input_label, inst_map, _, _ = pix2pixHD.encode_input(Variable(label), Variable(inst), infer=True)
    # sample clusters from precomputed features             
    #feat_map = pix2pixHD.sample_features(inst_map)
    input_concat = torch.cat((input_label, feat_map), dim=1)                        
    fake_image = pix2pixHD.netG.forward(input_concat)
    return fake_image

In [ ]:
# color stuff for plotting feature cluster centers
import random

def get_random_color(pastel_factor = 0.5):
    return [(x+pastel_factor)/(1.0+pastel_factor) for x in [random.uniform(0,1.0) for i in [1,2,3]]]

def color_distance(c1,c2):
    return sum([abs(x[0]-x[1]) for x in zip(c1,c2)])

def generate_new_color(existing_colors,pastel_factor = 0.5):
    max_distance = None
    best_color = None
    for i in range(0,100):
        color = get_random_color(pastel_factor = pastel_factor)
        if not existing_colors:
            return color
        best_distance = min([color_distance(color,c) for c in existing_colors])
        if not max_distance or best_distance > max_distance:
            max_distance = best_distance
            best_color = color
    return best_color

In [ ]:
# TODO
# Plot ALL the features for a given class, see if it forms a continuous 3D curve/clusters?

# for testing, get the cluster centers
clusters_path = os.path.join(opt.checkpoints_dir, opt.name, opt.cluster_path) 
features_clustered = np.load(clusters_path).item()

print(features_clustered)

extrema_clustered_path = os.path.join(opt.checkpoints_dir, opt.name, 'features_extrema_010.npy')
features_extrema_clustered = np.load(extrema_clustered_path).item()

#opt.cluster_path = "features_extrema_010.npy"
#features_clustered = features_extrema_clustered

fig = plt.figure(1)
ax = Axes3D(fig)
colors = [get_random_color()]


classes = ['building', 'road', 'pole']
for cat in classes:
    class_id = lab_to_id[cat]
    points = features_clustered[class_id]
    print(cat, points)
    colors.append(generate_new_color(colors))
    for pt in points:
        ax.scatter(*pt, color=colors[-1])
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


In [ ]:
def get_features(model, image, inst):
    return model.encode_features(image, inst)

In [ ]:
# cobblestone_image = cobblestone_data['image']
# print(cobblestone_data)
# cobblestone_inst = cobblestone_data['inst']
# cobblestone_features = get_features(model, cobblestone_image, cobblestone_inst)
# print(cobblestone_features)

In [ ]:


labels = im0['label'].clone()
instances = im0['inst'].clone()


# Just to test, attempt rewriting a semantic label as another to see what is generated!

# target = 21
# source = 7

# # target = torch.Tensor([target])
# # target = target.expand(labels.size())

# mask = labels == source
# labels[mask] = target

# mask = instances == source
# instances[mask] = target


input_label, inst_map, _, _ = model.encode_input(Variable(labels), Variable(instances), infer=True)
base_feat_map = model.sample_features(inst_map)
semantic_region = 'road'
semantic_region_id = lab_to_id[semantic_region]
mask = (instances == semantic_region_id).cuda()
mask = mask[0][0]


cat = semantic_region
class_id = lab_to_id[cat]
features_list = features_clustered[class_id]

# features_list = np.array([
#     [0.5, -0.1, 0.5],
#     [1.0, -1.0, 0.9],
#     [-1.0, 1.0, -1.0],
#     [ 10.0, -0.16616863,  0.06818795]
# ])

features_list = np.array([
    [0.40335375, -0.37605467, 0.53809792],
    [ 0.33693951,  0.04128251,  0.11454804],
    [ 0.72480464,  0.05364915,  0.40521583]
])

#features_list =(np.random.rand(20, 3) - 0.5)*10

for desired_feature in features_list:
    print(desired_feature)
    feat_map = base_feat_map.clone()
    feat_map_copy = feat_map.clone()

    # overwrite entire layer with each feature index, mask later
    for (i, val) in enumerate(desired_feature):
        val = torch.Tensor([val])
        val = val.expand(feat_map_copy[0,i].size())
        feat_map_copy[0, i] = val
    
    # mask feature map off this
    for (i, layer) in enumerate(feat_map[0]):
        layer[mask] = feat_map_copy[0, i][mask]
    #feat_map = base_feat_map
    
    result = custom_inference(model, labels, instances, feat_map)
    img_tensor = result.data[0]
    img = util.tensor2im(img_tensor)
    plt.figure(figsize=(20,10))
    imgplot = plt.imshow(img)

In [ ]:
def sample_features(model, labels, instances):
    input_label, inst_map, _, _ = model.encode_input(Variable(labels), Variable(instances), infer=True)
    return model.sample_features(inst_map)
        
# End Goal: 
# Take in Region name, Segmentation, Texture (as image and instance map for now),
# output image with texture applied to region
def pix2pixHD_retexture(region_name, label_orig, inst_orig, texture_image, texture_inst=None, model=None, previous_features=None):


    if model is None:
        # TODO instantiate pix2pixHD
        print("TODO")
        return
    
    # check texture image conforms to required pixel dimensions
    if texture_image.size()[2:] != label_orig.size()[2:]:
        print("Texture image has size: ", texture_image.size(),)
        print("Input label map has size: ", label_orig.size(), )
        print("Currently do not support mismatched pixel sizes")
        # TODO implement crop/resize to allow different sizes
        # TODO look into encoder architecture to see how it deals with size
        return
    
    # ID of region we want to remap texture of
    target_region_id = lab_to_id[region_name]
    
    
    # sample features for this label, inst
    if texture_inst is None:
        # create a single instance over the entire image of the required category
        # for feeding into the encoder... not sure how well this will work
        single_inst_value = torch.Tensor([target_region_id]).expand(inst_orig.size())
        texture_inst = single_inst_value
        
    texture_features = model.encode_features(texture_image, texture_inst)
    #print(texture_features)
    print("Taking feature for ID: ", target_region_id, "({})".format(region_name))
    target_feature = texture_features[target_region_id][0][:3]
    print("=> Target feature: ", target_feature)
    
    # just to make sure we don't modify anything in place accidentally
    labels = label_orig.clone()
    instances = inst_orig.clone()


    input_label, inst_map, _, _ = model.encode_input(Variable(labels), Variable(instances), infer=True)
    
    if previous_features is None:
        feat_map = model.sample_features(inst_map).clone()
    else:
        feat_map = previous_features.clone()
    
    input_concat = torch.cat((input_label, feat_map), dim=1)
    original_generated = model.netG.forward(input_concat)
    
    # get mask of target instances
    mask = (instances == target_region_id).cuda()
    mask = mask[0][0]
    
    # get copy of feature map
    feat_map_copy = feat_map.clone()

    # overwrite entire layer with each feature index, mask later
    for (i, val) in enumerate(target_feature):
        val = torch.Tensor([val])
        val = val.expand(feat_map_copy[0,i].size())
        feat_map_copy[0, i] = val
    
    # mask feature map off this
    for (i, layer) in enumerate(feat_map[0]):
        layer[mask] = feat_map_copy[0, i][mask]
    #feat_map = base_feat_map
    
    # concateate feature map and input labels
    input_concat = torch.cat((input_label, feat_map), dim=1)                        
    modified_generated = model.netG.forward(input_concat)

    return (original_generated, modified_generated)

In [ ]:
texture_opt = TrainOptions()
print(sys.argv)
tmp = sys.argv
sys.argv = [tmp[0]]
texture_opt = texture_opt.parse(save=False)
sys.argv = tmp
texture_opt.phase = 'texture' # retrieves correct folders automatically
texture_opt.instance_feat = True # put in features to steer generation
texture_opt.name = 'label2city_512p_feat'
texture_opt.dataroot = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/datasets/cityscapes/'
texture_opt.checkpoints_dir = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/checkpoints/'
texture_opt.results = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/results/'
texture_opt.no_flip = True

# get test images
texture_data_loader = CreateDataLoader(texture_opt)
texture_dataset = texture_data_loader.load_data()


In [ ]:
eval_opt = TestOptions()
print(sys.argv)
tmp = sys.argv
sys.argv = [tmp[0]]
eval_opt = eval_opt.parse(save=False)
sys.argv = tmp
eval_opt.phase = 'eval' # retrieves correct folders automatically
eval_opt.instance_feat = True # put in features to steer generation
eval_opt.name = 'label2city_512p_feat'
eval_opt.dataroot = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/datasets/cityscapes/'
eval_opt.checkpoints_dir = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/checkpoints/'
eval_opt.results = '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/pix2pixHD/results/'
eval_opt.no_flip = True

# get test images
eval_data_loader = CreateDataLoader(eval_opt)
eval_dataset = eval_data_loader.load_data()


In [ ]:
evals = list(enumerate(eval_dataset))
textures = list(enumerate(texture_dataset))

In [ ]:
def show_group(image_tensors, layout_shape, figsize=(20,10)):
    fig = plt.figure(figsize=figsize)
    for (i, im) in enumerate(image_tensors):
        ax = fig.add_subplot(*layout_shape, i+1)
        ax.set_title('')
        plt.imshow(util.tensor2im(im))
    plt.show()

In [ ]:
def show_im(image_tensor, normalize=2.0, figsize=(20,10)):
    # borrowed from util.tensor2im with normalization modification
    image_numpy = image_tensor.cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / normalize * 255.0
    image_numpy = np.clip(image_numpy, 0, 255)
    if image_numpy.shape[2] == 1:        
        image_numpy = image_numpy[:,:,0]
    img = image_numpy.astype(np.uint8)
    #img = util.tensor2im(image_tensor)
    plt.figure(figsize=figsize)
    imgplot = plt.imshow(img)

In [ ]:
for (i, data) in textures:
    show_im(data['image'][0], normalize=2.0, figsize=(10,5))
    #show_im(data['inst'][0], normalize=35)
    #show_im(data['label'][0], normalize=35)


In [ ]:
# texture_0_data = textures[4][1]
# eval_0_data = evals[0][1]
# result = pix2pixHD_retexture('road', 
#                     eval_0_data['label'],
#                     eval_0_data['inst'],
#                     texture_0_data['image'], 
#                     texture_inst=texture_0_data['inst'], 
#                     model=model)
# unretextured = result[0]
# retextured = result[1]
# show_im(unretextured.data[0])
# show_im(retextured.data[0])
# show_im(texture_0_data['image'][0])

for (i, eval_datum) in evals[0:2]:
    generator_features = sample_features(model, eval_datum['label'], eval_datum['inst'])
    for (j, texture_datum) in textures:
        texture_inst = texture_datum['inst']
        if 'yellow_brick' in texture_datum['path'][-1]:
            texture_inst = None
        result = pix2pixHD_retexture('road', 
                                     eval_datum['label'],
                                     eval_datum['inst'],
                                     texture_datum['image'], 
                                     texture_inst=texture_inst, 
                                     model=model,
                                    previous_features=generator_features)
        unretextured = result[0]
        retextured = result[1]
        show_group([unretextured.data[0], retextured.data[0], texture_datum['image'][0]],
                   layout_shape=(3,1),
                   figsize=(40,30)
                    )
#         show_im(unretextured.data[0])
#         show_im(retextured.data[0])
#         show_im(texture_datum['image'][0])
